In [1]:
import re
class Vocabulary:
    PAD_token = 0     # Used for padding short sentences
    SOS_token = 1     # Start-of-sentence token
    EOS_token = 2     # End-of-sentence token
    def __init__(self, name, sentence_trim=50):
        self.PAD_token = 0     # Used for padding short sentences
        self.SOS_token = 1     # Start-of-sentence token
        self.EOS_token = 2     # End-of-sentence token
        self.NEW_WORD = 3
        self.START_NUMBER = 4
        self.END_NUMBER = 5
        self.name = name
        self.word2index = { 
            "new_word" : self.NEW_WORD,
            "PAD" : self.PAD_token,
            "SOS" : self.SOS_token,
            "EOS" : self.EOS_token,
            "<num>" : self.START_NUMBER,
            "</num>" : self.END_NUMBER
        }
        self.word2count = {
            "new_word" : 1,
            "PAD" : 1,
            "SOS" : 1,
            "EOS" : 1,
            "<num>" : 1,
            "</num>" : 1
        }
        self.index2word = {
            self.PAD_token: "PAD",
            self.SOS_token: "SOS",
            self.EOS_token: "EOS",
            self.NEW_WORD : "new_word",
            self.START_NUMBER: "<num>",
            self.END_NUMBER: "</num>"
        }
        self.num_words = 6
        self.num_sentences = 0
        self.longest_sentence = 0
        self.sentence_number = []
        self.sentence_trim = sentence_trim

    def add_word(self, word):
        if word not in self.word2index:
            # First entry of word into vocabulary
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            # Word exists; increase word count
            self.word2count[word] += 1

    def add_sentence(self, sentence):
        sentence_len = 0
        sentence = sentence.replace('|',' | ').replace(',',' , ').replace(':',' : ').replace('\n','').strip()
        numbers = re.findall('([-\d\.]+)',sentence)
        formattedSentence = []
        for word in sentence.split(" "):
            if word in numbers:
                word = ["<num>"] + [ c for c in word ] + ["</num>"]
                for c in word:
                    formattedSentence.append(c)
            else:
                formattedSentence.append(word.lower())
        for word in formattedSentence:
            sentence_len += 1
            self.add_word(word)
        if sentence_len > self.longest_sentence:
            # This is the longest sentence
            self.longest_sentence = sentence_len
        # Count the number of sentences
        self.num_sentences += 1
        self.sentence_number.append(sentence_len)

    def to_word(self, index):
        return self.index2word[index]

    def to_index(self, word):
        return self.word2index[word.lower()]

    def sentence_to_index(self, sentence, padding=False):
        sentence = sentence.replace('|',' | ').replace(',',' , ').replace(':',' : ').replace('\n','').strip()
        numbers = re.findall('([-\d\.]+)',sentence)
        formattedSentence = []
        for word in sentence.split(" "):
            if word in numbers:
                    word = ["<num>"] + [ c for c in word ] + ["</num>"]
                    for c in word:
                            formattedSentence.append(c)
            else:
                    formattedSentence.append(word.lower())
        array = []
        array.append(self.SOS_token)
        for tok in formattedSentence:
            try:
                    array.append(self.to_index(tok))
            except:
                    #self.add_word(tok.text)
                    #array.append(self.to_index(tok.text))
                    array.append(self.to_index("new_word"))
        array.append(self.EOS_token)
        if padding:
            for i in range(len(array),self.sentence_trim):
                    array.append(self.PAD_token)
            return array[:self.sentence_trim]
        else:
            return array

# Modelo

In [2]:
import pickle
import random
import operator
import numpy as np
import pandas as pd
import random
import re
import time
import math

import torch
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
def save_to_file(objeto, nome_arquivo):
    with open(nome_arquivo, 'wb') as output:
        pickle.dump(objeto, output, pickle.HIGHEST_PROTOCOL)


def load_file(nome_arquivo):
    with open(nome_arquivo, 'rb') as input:
        objeto = pickle.load(input)
    return objeto

sub_path = './'

vocab = load_file(sub_path+'vocab.pkl')

src = load_file(sub_path+'src.pkl')
trg = load_file(sub_path+'trg.pkl')

srcVal = load_file(sub_path+'srcVal.pkl')
trgVal = load_file(sub_path+'trgVal.pkl')

### Model Architecture

In [4]:
######################################################################
# Seq2Seq Network using Transformer
# ---------------------------------
#
# Transformer is a Seq2Seq model introduced in `“Attention is all you
# need” <https://papers.nips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf>`__
# paper for solving machine translation tasks.
# Below, we will create a Seq2Seq network that uses Transformer. The network
# consists of three parts. First part is the embedding layer. This layer converts tensor of input indices
# into corresponding tensor of input embeddings. These embedding are further augmented with positional
# encodings to provide position information of input tokens to the model. The second part is the
# actual `Transformer <https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html>`__ model.
# Finally, the output of the Transformer model is passed through linear layer
# that gives unnormalized probabilities for each token in the target language.
#


from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = -1, 0, 1, 2

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)


######################################################################
# During training, we need a subsequent word mask that will prevent the model from looking into
# the future words when making predictions. We will also need masks to hide
# source and target padding tokens. Below, let's define a function that will take care of both.
#

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

### Configuration

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
######################################################################
# Let's now define the parameters of our model and instantiate the same. Below, we also
# define our loss function which is the cross-entropy loss and the optimizer used for training.
#
torch.manual_seed(0)

SRC_VOCAB_SIZE = vocab.num_words+1
TGT_VOCAB_SIZE = vocab.num_words+1
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)
model = transformer
criterion = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [7]:
def count_parameters(model):
        return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 12,856,983 trainable parameters


In [18]:
def greedy_decode(model, src, max_len, start_symbol=1):
    #print(src.shape)
    src = (src.reshape(max_len,1))
    #print(src.shape)
    num_tokens = len(src)
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool).to(device)
    memory = model.encode(src, src_mask).to(device)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):
        memory_mask = torch.zeros(ys.shape[0], memory.shape[0]).to(device).type(torch.bool)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                                    .type(torch.bool)).to(device)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.item()
        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == 2: # 2 == EOS
          break
    return ys

In [26]:
import gc

BATCHSIZE = 128

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
            yield lst[i:i + n]

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def train_bert(model, optimizer, criterion, src, trg):
    model.train()
    epoch_loss = 0
    it = 0
    for i in chunks(np.arange(src.shape[1]), BATCHSIZE):
        it += 1
        # As transformações matriciais do treinamento existem porque o tgt tem que prever x+1 ou seja, se a entrada é x, a saída é x+1.
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src[:,i].to(device), trg[:-1,i].to(device))
        output = model(
            src[:,i].to(device), trg[:-1,i].to(device), src_mask,
            tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask
        )
        optimizer.zero_grad()
        loss = criterion(output.view(-1, output.shape[-1]), trg[1:,i].to(device).view(-1)) # Loss da saída de X+1
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
        gc.collect()
        torch.cuda.empty_cache()
    return epoch_loss / BATCHSIZE

def evaluate_bert(model, criterion, src, trg):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i in chunks(np.arange(src.shape[1]), BATCHSIZE):
            src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src[:,i].to(device), trg[:-1,i].to(device))
            output = model(
                src[:,i].to(device), trg[:-1,i].to(device), src_mask,
                tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask
            )
            #loss = criterion(output.argmax(2).reshape(-1,1)[:,0], trg[i,:].to(device).reshape(-1,1)[:,0])
            loss = criterion(output.view(-1, output.shape[-1]), trg[1:,i].to(device).view(-1))
            epoch_loss += loss.item()
            gc.collect()
            torch.cuda.empty_cache()
    return epoch_loss / BATCHSIZE

In [10]:
torch.cuda.empty_cache()

In [11]:
src = src.T
trg = trg.T

In [12]:
srcVal = srcVal.T
trgVal = trgVal.T

In [13]:
src.shape, trg.shape, srcVal.shape, trgVal.shape

(torch.Size([60, 19914]),
 torch.Size([60, 19914]),
 torch.Size([60, 4968]),
 torch.Size([60, 4968]))

In [27]:
asw = greedy_decode(model, src[:,2].to(device), 60, 1)
' '.join( vocab.index2word[i] for i in asw.squeeze().cpu().numpy())

'SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS'

In [28]:
N_EPOCHS = 1000
CLIP = 1
path = './'
best_valid_loss = float('inf')

try: 
    model.load_state_dict(torch.load(path+'math-bert-model.pt'))
    print('Model Loaded Successfully!')
except:
    print('No model loaded, starting training from scratch.')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    print('Start training',epoch)
    train_loss = train_bert(model, optimizer, criterion, src, trg)
    print('Validating...',epoch)
    valid_loss = evaluate_bert(model, criterion, srcVal, trgVal)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), path+'math-bert-model.pt')

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |    Val. PPL: {math.exp(valid_loss):7.3f}')
    with open(path+"modelTrainingOutput.txt", "a") as textFile:
        textFile.write(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s\n')
        textFile.write(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}\n')
        textFile.write(f'\t Val. Loss: {valid_loss:.3f} |    Val. PPL: {math.exp(valid_loss):7.3f}\n')
    if epoch % 10 == 0:
        with torch.no_grad():
            rnd = np.random.randint(1,100)
            o = greedy_decode(model, srcVal[:,rnd-1:rnd].to(device),60, 1)
            query = ' '.join( vocab.index2word[i] for i in srcVal[:,rnd-1:rnd].squeeze().cpu().numpy())
            answer = ' '.join( vocab.index2word[i] for i in o.squeeze().cpu().numpy())
        print(f'Testing:\n')
        print(f'\t Query:\n\t'+query+'\n')
        print(f'\t Answer:\n\t'+answer+'\n')
        print(f'\n')
        with open(path+"modelTrainingOutput.txt", "a") as textFile:
            textFile.write(f'Testing:\n')
            textFile.write(f'\t Query:\n\t'+query+'\n')
            textFile.write(f'\t Answer:\n\t'+answer+'\n')
            textFile.write(f'\n')

Model Loaded Successfully!
Start training 0
Validating... 0
Epoch: 01 | Time: 2m 12s
	Train Loss: 2.349 | Train PPL:  10.474
	 Val. Loss: 0.188 |    Val. PPL:   1.206
Testing:

	 Query:
	SOS would you be willing to calculate this function :  <num> 3 </num> + <num> 1 </num> EOS PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD

	 Answer:
	SOS it's my belief that the answer could be :  <num> 1 0 </num> EOS



Start training 1
Validating... 1
Epoch: 02 | Time: 2m 41s
	Train Loss: 0.751 | Train PPL:   2.119
	 Val. Loss: 0.165 |    Val. PPL:   1.179
Start training 2
Validating... 2
Epoch: 03 | Time: 2m 54s
	Train Loss: 0.678 | Train PPL:   1.969
	 Val. Loss: 0.156 |    Val. PPL:   1.169
Start training 3
Validating... 3
Epoch: 04 | Time: 2m 59s
	Train Loss: 0.638 | Train PPL:   1.892
	 Val. Loss: 0.146 |    Val. PPL:   1.157
Start training 4
Validating... 4
Epoch: 05 | Time: 2m 5

KeyboardInterrupt: 